In [ ]:
%pip install requests biopython torch transformers sentencepiece protobuf

import requests
import json
from Bio import SeqIO
import io
import torch
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 17.3 MB/s  0:00:23m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 36.9 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 23.7 MB/s  0:00:15m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 37.9 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 39.7 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 20.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 18.8 MB/s  0:00:19m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 38.5 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 28.9 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/26

In [4]:
donor_profile = {
    "id": "Donor_001",
    "hla_a":["A*02:01", "A*24:02"],
    "hla_b": ["B*44:02", "B*51:01"],
    "hla_drb1":["DRB1*04", "DRB*11:04"]
}
recipient_profile = {
    "id": "Recipient_123",
    "hla_a": ["A*01:01", "A*02:01"],
    "hla_b": ["B*44:02", "B*08:01"],
    "hla_drb1": ["DRB1*04:05", "DRB1*15:01"]
}

def get_all_alleles(profile):
  alleles =[]
  hla_keys = ['hla_a', 'hla_b', 'hlb_drb1']

  for key in hla_keys:
    if key in profile:
      alleles.extend(profile[key])

  return alleles

donor_alleles=get_all_alleles(donor_profile)
recipient_alleles = get_all_alleles(recipient_profile)

print("✅ TASK 1 COMPLETE")
print("=" * 40)
print(f"Donor ID: {donor_profile['id']}")
print(f"Donor alleles: {donor_alleles}")
print()
print(f"Recipient ID: {recipient_profile['id']}")
print(f"Recipient alleles: {recipient_alleles}")
print("=" * 40)
print(f"Total donor alleles: {len(donor_alleles)}")
print(f"Total recipient alleles: {len(recipient_alleles)}")



✅ TASK 1 COMPLETE
Donor ID: Donor_001
Donor alleles: ['A*02:01', 'A*24:02', 'B*44:02', 'B*51:01']

Recipient ID: Recipient_123
Recipient alleles: ['A*01:01', 'A*02:01', 'B*44:02', 'B*08:01']
Total donor alleles: 4
Total recipient alleles: 4


In [5]:
import requests
import time
import os
import json
from typing import List, Dict, Tuple, Optional
from pathlib import Path
from Bio import SeqIO
import io
import re
import warnings
warnings.filterwarnings('ignore')

# Import the setup script helper
from setup_hla_database import load_hla_path

class HLASequenceCache:
    def __init__(self):
        self.cache_dir = Path.home() / ".cache" / "hai_def" if os.name != 'nt' else Path.home() / "AppData" / "Local" / "hai_def_cache"
        self.cache_dir.mkdir(parents=True, exist_ok=True)
        self.sequence_cache_file = self.cache_dir / "hla_sequence_cache.json"
        self.sequence_cache = {}
        self._load_or_create_cache()

    def _download_complete_hla_dataset(self) -> Dict:
        """Load HLA dataset from cached FASTA file."""
        # Use the cached file from setup script
        cached_fasta_path = load_hla_path()
        
        if not cached_fasta_path:
            print("❌ HLA database not found in cache.")
            print("   Please run: python setup_hla_database.py")
            return {}
        
        if not cached_fasta_path.exists():
            print(f"❌ Cached FASTA file not found at: {cached_fasta_path}")
            return {}
        
        print(f"📥 Loading HLA FASTA database from cache: {cached_fasta_path}")
        try:
            with open(cached_fasta_path, 'r') as f:
                fasta_content = f.read()
            dataset = self._parse_fasta_to_dataset(fasta_content)
            if dataset:
                print(f"✅ Parsed {len(dataset)} unique HLA protein sequences from cached file")
                return dataset
            else:
                print("❌ Failed to parse FASTA content.")
                return {}
        except Exception as e:
            print(f"❌ Error reading cached FASTA file: {e}")
            return {}

    def _parse_fasta_to_dataset(self, fasta_content: str) -> Dict:
        """Parse FASTA content into dataset, storing the longest sequence for broader allele groups."""
        print("  Parsing FASTA file...")

        dataset = {}
        fasta_io = io.StringIO(fasta_content)

        for record in SeqIO.parse(fasta_io, "fasta"):
            header = record.description
            sequence = str(record.seq)

            # Skip if sequence is too short to be considered 'real'
            if len(sequence) < 100:
                continue

            allele_name = self._extract_allele_name(header)

            if allele_name:
                # Store the full allele name and its sequence
                if allele_name not in dataset or len(sequence) > len(dataset[allele_name]):
                    dataset[allele_name] = sequence

                # Handle broader allele groups (e.g., DRB1*04:01, DRB1*04)
                if '*' in allele_name:
                    parts = allele_name.split('*')
                    locus_prefix = parts[0]
                    allele_fields = parts[1].split(':')

                    # Store 3-field version (e.g., A*02:01:01)
                    if len(allele_fields) >= 3:
                        simplified_3_field = f"{locus_prefix}*{allele_fields[0]}:{allele_fields[1]}:{allele_fields[2]}"
                        if simplified_3_field not in dataset or len(sequence) > len(dataset[simplified_3_field]):
                            dataset[simplified_3_field] = sequence

                    # Store 2-field version (e.g., A*02:01)
                    if len(allele_fields) >= 2:
                        simplified_2_field = f"{locus_prefix}*{allele_fields[0]}:{allele_fields[1]}"
                        if simplified_2_field not in dataset or len(sequence) > len(dataset[simplified_2_field]):
                            dataset[simplified_2_field] = sequence

                    # Store 1-field version (e.g., A*02)
                    simplified_1_field = f"{locus_prefix}*{allele_fields[0]}"
                    if simplified_1_field not in dataset or len(sequence) > len(dataset[simplified_1_field]):
                        dataset[simplified_1_field] = sequence

        return dataset

    def _extract_allele_name(self, header: str) -> Optional[str]:
        """Extract HLA allele name from FASTA header using a robust regex."""
        allele_name_pattern = r'([A-Z0-9]+\*\d+(?::\d+){0,3})'

        match = re.search(allele_name_pattern, header)

        if match:
            extracted_allele = match.group(1)
            if '*' in extracted_allele and re.match(r'^[A-Z0-9]+\*\d+(?::\d+){0,3}$', extracted_allele):
                return extracted_allele
        return None

    def _build_sequence_cache(self, dataset: Dict) -> Dict:
        """Build optimized sequence cache."""
        print("🔨 Building optimized sequence cache...")

        sequence_cache = {}

        for allele_name, sequence in dataset.items():
            if sequence and len(sequence) > 100:
                sequence_cache[allele_name] = sequence

        print(f"✅ Cache built with {len(sequence_cache)} sequences")
        return sequence_cache

    def _load_or_create_cache(self):
        """Load cache or create new."""
        if self.sequence_cache_file.exists():
            try:
                with open(self.sequence_cache_file, 'r') as f:
                    self.sequence_cache = json.load(f)
                print(f"✅ Loaded cached sequences: {len(self.sequence_cache)} sequences")
                return
            except:
                print("⚠️ Cache corrupted, rebuilding...")

        print("📥 Building sequence cache from FASTA...")
        dataset = self._download_complete_hla_dataset()
        if dataset:
            self.sequence_cache = self._build_sequence_cache(dataset)
            if self.sequence_cache:
                with open(self.sequence_cache_file, 'w') as f:
                    json.dump(self.sequence_cache, f, indent=2)
                print(f"💾 Saved sequence cache to {self.sequence_cache_file}")

    def get_sequence(self, allele_name: str) -> str:
        """Get sequence with multiple fallback strategies."""
        if allele_name in self.sequence_cache:
            seq = self.sequence_cache[allele_name]
            if len(seq) > 100:
                return seq

        api_seq = self._fetch_from_api(allele_name)
        if api_seq and len(api_seq) > 100:
            self.sequence_cache[allele_name] = api_seq
            self.save_cache()
            return api_seq

        return self._get_dummy_sequence(allele_name)

    def _fetch_from_api(self, allele_name: str) -> Optional[str]:
        """API fallback for individual alleles."""
        encoded_allele = allele_name.replace(':', '%3A')

        try:
            search_url = f"https://www.ebi.ac.uk/cgi-bin/ipd/api/allele?name={encoded_allele}&release=latest"
            search_response = requests.get(search_url, timeout=15)

            if search_response.status_code != 200:
                return None

            search_data = search_response.json()

            accession = None
            if isinstance(search_data, dict) and 'data' in search_data and isinstance(search_data['data'], list) and len(search_data['data']) > 0:
                for item in search_data['data']:
                    item_name = item.get('name', '')
                    if item_name.startswith(allele_name):
                        accession = item.get('accession')
                        break

            if not accession:
                return None

            allele_detail_url = f"https://www.ebi.ac.uk/cgi-bin/ipd/api/allele/{accession}"
            detail_response = requests.get(allele_detail_url, timeout=15)

            if detail_response.status_code == 200:
                detail_data = detail_response.json()

                if (isinstance(detail_data, dict) and
                    'sequence' in detail_data and
                    'protein' in detail_data['sequence']):

                    protein_seq = detail_data['sequence']['protein']

                    if protein_seq and len(protein_seq) > 100:
                        return protein_seq

        except requests.exceptions.RequestException:
            pass
        except (json.JSONDecodeError, KeyError, IndexError, ValueError):
            pass
        except Exception:
            pass

        return None

    def _get_dummy_sequence(self, allele_name: str) -> str:
        """Get realistic dummy sequence."""
        dummy_sequences = {
            'A': "MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF_DUMMY_SEQ",
            'B': "MVHWTAEEKQLITGLWGKVNVAECGAEALARLLIVYPWTQRFFASF_DUMMY_SEQ",
            'DRB1': "MVCLKLPGGSYMAKLTVQIPSSVFAVTSILMRSSAVLGLLLLGAG_DUMMY_SEQ",
        }

        if allele_name.startswith('A'):
            return dummy_sequences['A']
        elif allele_name.startswith('B'):
            return dummy_sequences['B']
        elif allele_name.startswith('DRB1'):
            return dummy_sequences['DRB1']
        else:
            return "UNKNOWN_LOCUS_DUMMY_SEQ"

    def save_cache(self):
        """Save cache to disk."""
        try:
            with open(self.sequence_cache_file, 'w') as f:
                json.dump(self.sequence_cache, f, indent=2)
        except Exception as e:
            print(f"Error saving cache: {e}")

# ============================================
# TASK 3: FETCH ALL SEQUENCES
# ============================================

def fetch_all_sequences(allele_list: List[str], hla_cache_instance) -> Dict[str, str]:
    """Fetch sequences using cache."""
    sequences = {}

    print(f"\n📥 Fetching {len(allele_list)} sequences...")

    for i, allele in enumerate(allele_list, 1):
        print(f"  [{i}/{len(allele_list)}] Fetching {allele}", end=": ")
        seq = hla_cache_instance.get_sequence(allele)

        if seq and len(seq) > 100: # Check for real sequence based on length
            sequences[allele] = seq
            print(f"✅ Real ({len(seq)} aa)")
        else:
            sequences[allele] = seq # Store dummy too, but mark as failed
            print(f"❌ Dummy ({len(seq)} aa)")

    return sequences

# Your alleles from Task 1 (re-define for execution safety if not in scope)
if 'donor_alleles' not in locals() or 'recipient_alleles' not in locals():
    donor_profile = {
        "id": "Donor_001",
        "hla_a":["A*02:01", "A*24:02"],
        "hla_b": ["B*44:02", "B*51:01"],
        "hla_drb1":["DRB1*04", "DRB1*11:04"]
    }
    recipient_profile = {
        "id": "Recipient_123",
        "hla_a": ["A*01:01", "A*02:01"],
        "hla_b": ["B*44:02", "B*08:01"],
        "hla_drb1": ["DRB1*04:05", "DRB1*15:01"]
    }

    def get_all_alleles(profile):
        alleles =[]
        hla_keys = ['hla_a', 'hla_b', 'hla_drb1']
        for key in hla_keys:
            if key in profile:
                alleles.extend(profile[key])
        return alleles

    donor_alleles=get_all_alleles(donor_profile)
    recipient_alleles = get_all_alleles(recipient_profile)


print("\n" + "="*70)
print("🚀 TASK 3: FETCHING ALL SEQUENCES")
print("="*70)

hla_cache = HLASequenceCache()

print("\n1. Donor sequences:")
donor_sequences = fetch_all_sequences(donor_alleles, hla_cache)

print("\n2. Recipient sequences:")
recipient_sequences = fetch_all_sequences(recipient_alleles, hla_cache)

# Summary
def analyze_sequences(seq_dict, name):
    real_count = sum(1 for s in seq_dict.values() if len(s) > 100)
    dummy_count = len(seq_dict) - real_count
    print(f"\n{name} summary: {real_count} real sequences, {dummy_count} dummy sequences.")

analyze_sequences(donor_sequences, "Donor")
analyze_sequences(recipient_sequences, "Recipient")

# Save sequences to files
def save_sequences_to_fasta(sequences_dict: Dict[str, str], filename_base: str):
    """Save sequences in FASTA format."""
    filepath = Path.cwd() / filename_base
    with open(filepath, 'w') as f:
        for allele, sequence in sequences_dict.items():
            f.write(f">{allele}\n")
            for i in range(0, len(sequence), 60):
                f.write(sequence[i:i+60] + "\n")
            f.write("\n")
    print(f"✓ Saved to {filepath}")

save_sequences_to_fasta(donor_sequences, "donor_sequences.fasta")
save_sequences_to_fasta(recipient_sequences, "recipient_sequences.fasta")

print("\n" + "="*70)
print("✅ TASK 3 COMPLETE!")
print("   HLA sequences fetched from cached database.")
print("="*70)



🚀 TASK 3: FETCHING ALL SEQUENCES
✅ Loaded cached sequences: 44726 sequences

1. Donor sequences:

📥 Fetching 4 sequences...
  [1/4] Fetching A*02:01: ✅ Real (365 aa)
  [2/4] Fetching A*24:02: ✅ Real (365 aa)
  [3/4] Fetching B*44:02: ✅ Real (362 aa)
  [4/4] Fetching B*51:01: ✅ Real (362 aa)

2. Recipient sequences:

📥 Fetching 4 sequences...
  [1/4] Fetching A*01:01: ✅ Real (365 aa)
  [2/4] Fetching A*02:01: ✅ Real (365 aa)
  [3/4] Fetching B*44:02: ✅ Real (362 aa)
  [4/4] Fetching B*08:01: ✅ Real (362 aa)

Donor summary: 4 real sequences, 0 dummy sequences.

Recipient summary: 4 real sequences, 0 dummy sequences.
✓ Saved to /workspaces/Donor-Recipient-Compatibility/donor_sequences.fasta
✓ Saved to /workspaces/Donor-Recipient-Compatibility/recipient_sequences.fasta

✅ TASK 3 COMPLETE!
   HLA sequences fetched from cached database.
